In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
# add . to module name
import sys
sys.path.append('../src/')

In [41]:
from package.logger import setup
setup("INFO")

In [45]:
import pandas as pd
import os
from package.logger import llog, Timed
from package import storage, osm
from package.footpaths import (
    get_osm_path_from_city_id,
    download_osm,
    create_nearby_stops_map,
    create_nx_graph,
    add_nearest_node_to_stops,
)
from package.graph import igraph


In [47]:
city_id = "Koeln"
osm_path = get_osm_path_from_city_id(city_id)
stops_path = "../data/cleaned/stops.csv"
avg_walking_speed = 1.4  # m/s
max_walking_duration = 10 * 60  # s

if not os.path.exists(osm_path) and city_id:
	llog.info("Downloading OSM data")
	download_osm(city_id, osm_path)
else:
	llog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

with Timed.info("Reading OSM network"):
	# TODO: optimally we should first crop the data and then read the network
	nodes, edges = osm.read_network(osm_reader)

with Timed.info("Reading stops"):
	stops_df = storage.read_gdf(stops_path)

with Timed.info("Cropping OSM network to stops"):
	nodes, edges = osm.crop_to_stops(nodes, edges, stops_df)

with Timed.info("Creating networkx graph"):
	nx_graph = create_nx_graph(osm_reader, nodes, edges)

with Timed.info("Adding nearest network node to each stop"):
	stops_df = add_nearest_node_to_stops(stops_df, nx_graph)

with Timed.info("Finding potential nearby stops for each stop"):
	nearby_stops_map = create_nearby_stops_map(
		stops_df, avg_walking_speed, max_walking_duration
	)


[12:26:46] INFO     Using existing OSM data                                                         ]8;id=746713;file:///tmp/ipykernel_73857/366977008.py\366977008.py]8;;\:]8;id=756976;file:///tmp/ipykernel_73857/366977008.py#11\11]8;;\

           INFO     Reading OSM network                                                             ]8;id=553565;file:///tmp/ipykernel_73857/366977008.py\366977008.py]8;;\:]8;id=792930;file:///tmp/ipykernel_73857/366977008.py#15\15]8;;\

[12:27:44] INFO     Reading OSM network done (57.85 seconds)                                        ]8;id=245876;file:///tmp/ipykernel_73857/366977008.py\366977008.py]8;;\:]8;id=134327;file:///tmp/ipykernel_73857/366977008.py#15\15]8;;\

           INFO     Reading stops                                                                   ]8;id=553699;file:///tmp/ipykernel_73857/366977008.py\366977008.py]8;;\:]8;id=69548;file:///tmp/ipykernel_73857/366977008.py#19\19]8;;\

           INFO     Reading stops done (0.04 seconds)                                               ]8;id=185537;file:///tmp/ipykernel_73857/366977008.py\366977008.py]8;;\:]8;id=297998;file:///tmp/ipykernel_73857/366977008.py#19\19]8;;\

...

           INFO     Calculating distances between nearby stops                                      ]8;id=774553;file:///tmp/ipykernel_73857/366977008.py\366977008.py]8;;\:]8;id=739805;file:///tmp/ipykernel_73857/366977008.py#51\51]8;;\

           INFO     Creating igraph graph                                                              ]8;id=153743;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/graph/igraph.py\igraph.py]8;;\:]8;id=911461;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/graph/igraph.py#20\20]8;;\

[12:27:52] INFO     Creating igraph graph done (2.61 seconds)                                          ]8;id=768128;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/graph/igraph.py\igraph.py]8;;\:]8;id=221245;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/graph/igraph.py#20\20]8;;\

           INFO     Calculating distances between nearby stops failed (2.76 seconds)                ]8;id=232546;file:///tmp/ipykernel_73857/366977008.py\366977008.py]8;;\:]8;id=143372;file:///tmp/ipykernel_73857/366977008.py#51\51]8;;\

TypeError: 'zip' object is not subscriptable

In [48]:

stop_to_node_map: dict[str, int] = stops_df.set_index("stop_id")[
	"nearest_node"
].to_dict()
node_to_stop_map: dict[int, str] = stops_df.set_index("nearest_node")[
	"stop_id"
].to_dict()

# this map contains the one-to-many queries that have to be solved on the graph
source_targets_map: dict[int, list[int]] = {
	stop_to_node_map[stop_id]: [
		stop_to_node_map[stop_id] for stop_id in nearby_stops
	]
	for stop_id, nearby_stops in nearby_stops_map.items()
}

with Timed.info("Calculating distances between nearby stops"):
	source_targets_distance_map = igraph.query_multiple_one_to_many(
		source_targets_map, osm_reader, nodes, edges
	)

footpaths: dict[str, dict[str, int]] = {}
for source_node, targets_distance_map in source_targets_distance_map.items():
	stop_id = node_to_stop_map[source_node]
	footpaths[stop_id] = {  # type: ignore
		node_to_stop_map[target_node]: distance / avg_walking_speed
		for target_node, distance in targets_distance_map.items()
	}

[12:29:16] INFO     Calculating distances between nearby stops                                     ]8;id=837471;file:///tmp/ipykernel_73857/4088830788.py\4088830788.py]8;;\:]8;id=969346;file:///tmp/ipykernel_73857/4088830788.py#16\16]8;;\

           INFO     Creating igraph graph                                                              ]8;id=639388;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/graph/igraph.py\igraph.py]8;;\:]8;id=316645;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/graph/igraph.py#20\20]8;;\

[12:29:25] INFO     Creating igraph graph done (8.86 seconds)                                          ]8;id=289517;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/graph/igraph.py\igraph.py]8;;\:]8;id=820474;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/graph/igraph.py#20\20]8;;\

  0%|          | 0/97 [00:00<?, ?it/s]

[12:29:28] INFO     Calculating distances between nearby stops done (11.52 seconds)                ]8;id=454568;file:///tmp/ipykernel_73857/4088830788.py\4088830788.py]8;;\:]8;id=928845;file:///tmp/ipykernel_73857/4088830788.py#16\16]8;;\

In [12]:
s_edges = edges[['u', 'v', 'length']]

In [13]:
s_edges.to_csv("../tmp/edges.csv", index=False)

In [14]:
import pandas as pd
s_edges = pd.read_csv("../tmp/edges.csv")

In [1]:
import mcr_py
mcr_py.query_multiple_one_to_many

<function mcr_py.mcr_py.query_multiple_one_to_many(raw_edges, source_target_nodes_map)>